# Credmark Modeling Framework Example for Jupyter notebook
## Introduction

version: 2023.4.15

## Installation

If you would like to use Conda environment, run below commands in the terminal.

    NAME=cmk # rename it to what you want to
    conda create -y --name "$NAME"
    conda install -y --name "$NAME" pip
    conda activate ${NAME}
    conda install -y --name "$NAME" ipython jupyterlab nb_conda_kernels black isort
    conda install -y --name "$NAME" jupyterlab_code_formatter ipywidgets
    conda install -y --name "$NAME" scipy numpy
    conda install -y --name "$NAME" altair pandas matplotlib openpyxl xlsxwriter
    conda activate ${NAME}

Launch Jupyter notebook and change the kernel in the newly created environment and run following commands.

They will download the git repository of Cremark models in the directory two-levels up `../../` which is on the same level of `credmark-models-notebook` (line 1) and install the Credmark Modeling Framework (line 2).

    !cd ../../ && git clone https://github.com/credmark/credmark-models-py.git
    !cd ../../credmark-models-py && pip install -r requirements.txt && pip install -e .


## Initialize

### 1. Create context and some shortcuts for frequently used utilities.

* Create the initialization parameters
    - chain_id: 1 for ETH
    - block_number: None or a specific number.
    - chain_to_provider_url: Web3 node (archive is preferred)
    - api_url: None or a specific gateway server address
    - use_local_models: None, '-' '*', or a comma-separated list.
    - register_utility_global: True (set global variables for various utilities, e.g. ledger, web3, etc.)

- Method 1: use ipython magic

Use default parameter

```python
%reload_ext credmark.cmf.ipython
context, model_loader = %cmf default
```

You may turn on verbose to show the initialization parameters.

```python
%reload_ext credmark.cmf.ipython
context, model_loader = %cmf default -v
```

Or, with specific parameters

```python
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True
    }

context, model_loader = %cmf cmf_param
```

- Method 2: use function.

```python
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True
    }

context, _model_loader = create_cmf(cmf_param)
```

<div class="alert alert-block alert-info">
    <b>Note:</b> Leave `chain_to_provider_url` unset to use Credmark's nodes.
</div>

In [1]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

* Get help for Cmf extension

In [2]:
%cmf help

Example:
%reload_ext credmark.cmf.ipython
param = {'chain_id': 1,
 'block_number': None,
 'model_loader_path': [],
 'chain_to_provider_url': {},
 'api_url': None,
 'use_local_models': None,
 'register_utility_global': True}
context, model_loader = %cmf param
# or
%cmf param
context, model_loader = _
Other commands:
- %cmf param -v: verbose
- %cmf default_param: returns default parameters
Example: param = %cmf default_param
- %cmf default: setup with default parameters
Example: context, model_loader = %cmf default
- %cmf help: get help
- %cmf help_param: get help for parameters



In [3]:
%cmf help_param

Doc:

    Cmf Initialization Parameters

    :param chain_id: Chain id, default to 1
    :param block_number: (Optional) None or int
    :param model_loader_path: List of path to the models directories
    :param chain_to_provider_url: A dictionary mapping chain ID to node RPC URL, e.g. {'1': 'http://192.168.68.122:10444'}
    :param api_url: (Optional) None or URL to Credmark gateway
    :param use_local_models: None (top-level models run local), '*' (all model run locally), or '-' (all models run remotely), or a comma-separated list of models
    :param register_utility_global: True (register global variables for utilities like ledger, default) or False
    


In [4]:
%cmf default_param

{'chain_id': 1,
 'block_number': None,
 'model_loader_path': [],
 'chain_to_provider_url': {},
 'api_url': None,
 'use_local_models': None,
 'register_utility_global': True}

### 2. (Optional) adjust logging level

You could change the level of logging with `log_output()`.

In [5]:
import logging
# Change output of logging to a file, and 
# Change the level to DEBUG when no level is specified.
# logging_output('../tmp/debug.log')

# Change level of logging from the default (WARNING) to
# either INFO (less information)
# or DEBUG(more information).
# Still output to stream.
log_output(log_level=logging.INFO)

2023-04-17 14:38:46,996 - credmark.cmf.engine.context - INFO - Enabled log with level=INFO


## Use Cmf

### 1. Basic utilities

In [6]:
get_dt(2022, 5, 3)

datetime.datetime(2022, 5, 3, 0, 0, tzinfo=datetime.timezone.utc)

In [7]:
get_block(get_dt(2022, 5, 3))

14701368

In [8]:
context.block_number

17064589

In [9]:
context.chain_id

1

In [10]:
context.web3

In [11]:
context.web3.eth.get_balance('0xd3CdA913deB6f67967B99D67aCDFa1712C293601')

2961254664753178106

In [12]:
context.web3.eth.get_block(context.block_number)['gasUsed']

17327311

### 2. Use models

1. Type `models.` and hit `TAB` key, a list of models will be pop up.
2. Select a model and type ? will show description and help, `models.token.info?`
3. Show the list of models wiht `dir(models)`, or `model_loader.loaded_model_versions()` with version information.
4. If you have updated any model in the folder on disk, run `model_loader.reload()` to refresh.

In [13]:
dir(models)

['aave_v2.account_info',
 'aave_v2.account_info_reserve',
 'aave_v2.account_summary',
 'aave_v2.account_summary_historical',
 'aave_v2.assets',
 'aave_v2.get_incentive_controller',
 'aave_v2.get_lending_pool',
 'aave_v2.get_lending_pool_provider',
 'aave_v2.get_lending_pool_providers_from_registry',
 'aave_v2.get_lp_reward',
 'aave_v2.get_oracle_price',
 'aave_v2.get_price_oracle',
 'aave_v2.get_protocol_data_provider',
 'aave_v2.get_staking_reward',
 'aave_v2.lending_pool_assets',
 'aave_v2.lending_pool_assets_portfolio',
 'aave_v2.overall_liabilities_portfolio',
 'aave_v2.reserve_config',
 'aave_v2.token_asset',
 'aave_v2.token_liability',
 'account.portfolio',
 'account.token_historical',
 'account.token_return',
 'account.token_return_historical',
 'account.token_transfer',
 'account.var',
 'accounts.portfolio',
 'accounts.token_historical',
 'accounts.token_historical_balance',
 'accounts.token_return',
 'accounts.token_return_historical',
 'accounts.token_transfer',
 'accounts.va

In [14]:
models.tokens.erc20?

Call signature:
models.tokens.erc20(
    input: Union[pydantic.main.BaseModel, credmark.dto.IntDTO, credmark.dto.StrDTO, credmark.dto.FloatDTO, dict, NoneType] = None,
    return_type: Union[dict, Type[Union[pydantic.main.BaseModel, credmark.dto.IntDTO, credmark.dto.StrDTO, credmark.dto.FloatDTO]], NoneType] = None,
    version: Optional[str] = None,
    **kwargs,
) -> Union[dict, pydantic.main.BaseModel, credmark.dto.IntDTO, credmark.dto.StrDTO, credmark.dto.FloatDTO]
Type:           RunModelMethod
String form:    <credmark.cmf.model.models.RunModelMethod object at 0x7f8368fe79d0>
File:           ~/dev/credmark/credmark-model-framework-py/credmark/cmf/model/models.py
Docstring:     
Run a model.

Available models: 


In [15]:
model_loader.reload()

## 3. Create and use types

### 3.1 Example - get price for USDC

In [16]:
usdt = Token(address='0xdAC17F958D2ee523a2206206994597C13D831ec7')
print('USDT decimals:', usdt.decimals)
models.chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

USDT decimals: 6


{'price': 1.001,
 'src': 'chainlink.price-by-feed|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|None|t:52560s|r:0'}

In [17]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

{'src': 'chainlink.price-by-feed|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|None|t:17407s|r:0',
 'price': 1.00010233}

In [18]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_registry(base=usdt)

{'src': 'chainlink.price-by-registry|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|True|t:17407s|r:0',
 'price': 1.00010233,
 'quoteAddress': '0x0000000000000000000000000000000000000348'}

### 3.1 Run a model

#### Run as of current block

In [19]:
aave = Token(symbol='AAVE')

print(block_number,
      block_number.timestamp_datetime,
      'models:', context.models.price.quote(base=aave, return_type=Price).price,
      'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price).price)

17064589 2023-04-17 05:59:59+00:00 models: 82.0718795 run_model: 82.0718795


#### Run as of past blocks

In [20]:
context.block_number.timestamp_datetime

datetime.datetime(2023, 4, 17, 5, 59, 59, tzinfo=datetime.timezone.utc)

In [21]:
for dt in [get_dt(2022, 5, 3), get_dt(2022, 5, 9), get_dt(2022, 1, 1)]:
    block = get_block(dt)
    print(block,
          block.timestamp_datetime,
          'models:', context.models(block).price.quote(base=aave, return_type=Price).price,
          'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price, block_number=block).price)

14701368 2022-05-02 23:59:50+00:00 models: 144.94887195 run_model: 144.94887195
14739154 2022-05-08 23:59:54+00:00 models: 125.63830024 run_model: 125.63830024
13916165 2021-12-31 23:59:49+00:00 models: 255.01691343 run_model: 255.01691343


### 3.2 Travel between different blocks

Tavel is one-way only - can only travel to earlier/smaller block numbers, and not to the future/later blocks.

In [22]:
context.block_number = get_block(get_dt(2022, 5, 3))
models.price.quote(base=aave, return_type=Price).price,

(144.94887195,)

In [23]:
# Below will gave any error as we wants to travel back
# context.block_number = get_block(get_dt(2022, 5, 30))
# models.chainlink.price_usd(input=aave, return_type=Price).price

Instead, create a new context. It will automatically become the current context.

In [24]:
cmf_params_old = cmf_param | {'block_number': get_block(get_dt(2022, 4, 3))}

context_old,_ = %cmf cmf_params_old
context_old.block_number.timestamp_datetime, context.block_number.timestamp_datetime

{'chain_id': 1, 'block_number': 14509821, 'api_url': None, 'use_local_models': None, 'register_utility_global': True, 'chain_to_provider_url': {'chain_to_provider_url': {'1': 'http://1.cmk.chaindeck.io:31288/257452059af80bc3806ba9ba1eb67bb4c830b3125c33b804', '1_1': 'ws://localhost:10445', '1_3': 'https://h2.runchee.com', '1_2': 'http://192.168.68.122:10444', '1_7': 'http://35.81.151.209/f1c5ae2084b7f3b3a566024f', '1_4': 'http://ap303:10444', '1_5': 'https://h2.runchee.com/265c0b59-5d55-4ee2-ad9f-83715d4b2249', '1_8': 'http://192.168.68.117:8545', '1_9': 'http://192.168.68.122:8546', '1_10': 'http://10.30.30.2:8545', '56': 'https://stylish-tame-thunder.bsc.quiknode.pro/23c18aa1188026b63de8ee99e9a55ff7b6389180/', '137': 'http://137.cmk.chaindeck.io:30195/257452059af80bc3806ba9ba1eb67bb4c830b3125c33b806', '10': 'https://opt-mainnet.g.alchemy.com/v2/GNMH2cUrYfCXOBtEBYju5gPxt8RKDVu7', '42161': 'https://arb-mainnet.g.alchemy.com/v2/CZoj8IDvdcwhbIdkZpvc_kIeIUi91uOd'}, '1': 'https://nodes.cred

(datetime.datetime(2022, 4, 2, 23, 59, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 5, 2, 23, 59, 50, tzinfo=datetime.timezone.utc))

If we would like to use previously defined context, run `context.set_current()` to set it back to the current context.

Otherwise, we may encounter error with running model for a block number later than the context.

In [25]:
context.set_current()

### 4. Ledger

In [26]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
             .to_dataframe())
df_ledger

,block_hash,from_address,to_address,value
0,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x8c6f71f96419325c2f637332650029e2a53787a4,0xd19053361e356e79166b22beaacd248b1b63135f,5294873127907440
1,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xcad621da75a66c7a8f4ff86d30a2bf981bfc8fdd,0x8680aa8afc73f65d80d6561bcced49a7da5e2568,1050486160000000000
2,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x2ef2284a784c1fa06f95919fab978e87da18bc66,0x2ef2284a784c1fa06f95919fab978e87da18bc66,0
3,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xeb2629a2734e272bcc07bda959863f316f4bd4cf,0x8a2360d34628836cafe46026efd9e8bf1ae23f02,42004000000000000
4,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xf16e9b0d03470827a95cdfd0cb8a8a3b46969b91,0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c,0


In [27]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
        .to_dataframe())
df_ledger

,block_hash,from_address,to_address,value
0,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x8c6f71f96419325c2f637332650029e2a53787a4,0xd19053361e356e79166b22beaacd248b1b63135f,5294873127907440
1,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xcad621da75a66c7a8f4ff86d30a2bf981bfc8fdd,0x8680aa8afc73f65d80d6561bcced49a7da5e2568,1050486160000000000
2,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x2ef2284a784c1fa06f95919fab978e87da18bc66,0x2ef2284a784c1fa06f95919fab978e87da18bc66,0
3,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xeb2629a2734e272bcc07bda959863f316f4bd4cf,0x8a2360d34628836cafe46026efd9e8bf1ae23f02,42004000000000000
4,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xf16e9b0d03470827a95cdfd0cb8a8a3b46969b91,0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c,0
